## Dependencias

In [1]:
import numpy as np
import requests 
import os
import shutil
import pandas as pd
from glom import glom
from multiprocessing import Pool
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [14]:
def make_pool(func, params, threads):
    """
    Ejecuta de forma simultánea múltiples llamadas a una función
    :param func: function, objeto función a paralelizar
    :param params: iterable, parámetros de evaluación paralela
    :param threads: int, número de hilos de multiprocesamiento
    :return: list, resultado de la ejecución paralela agrupada en una lista
    """
    pool = Pool(threads)
    data = pool.starmap(func, params)
    pool.close()
    pool.join()
    del pool
    return [x for x in data]

In [23]:
def crear_carpeta_en_datos(numero:int):
    """ Crea una carpeta en la carpeta datos con el nombre del numero que se le pasa como parametro.

    Args:
        numero (int): Numero de la carpeta a crear
    """
    # Directorio base
    directorio_base = "datos"
    
    # Ruta completa de la carpeta a crear
    nueva_carpeta = os.path.join(directorio_base, f"{numero:03d}")
    
    try:
        # Si la carpeta ya existe, la borramos completamente
        if os.path.exists(nueva_carpeta):
            shutil.rmtree(nueva_carpeta)
        
        # Creamos la carpeta nuevamente
        os.makedirs(nueva_carpeta)
    except Exception as e:
        print(f"Error al crear la carpeta: {e}")


In [24]:
def obtener_datos_pokemon(id: int) -> pd.DataFrame:
    url = f"https://pokeapi.co/api/v2/pokemon/{id}"
    req = requests.get(url)
    datos = {}
    if req.status_code == 200:
        payload = req.json()
        # Extracción de claves directas
        claves = ['base_experience',
                  'height',
                  'name',
                  'sprites.other.official-artwork.front_default']
        datos.update(
            dict(zip(claves, map(lambda x: glom(payload, x), claves))))
        datos["image"] = datos.pop(
            "sprites.other.official-artwork.front_default")
        # Extracción de stats
        stats = glom(payload, 'stats')

        def getStat(stat):
            return (glom(stat, 'stat.name'), glom(stat, 'base_stat'))
        stats = dict(map(getStat, stats))
        datos.update(stats)
        datos = pd.Series(datos).to_frame().T
        datos.insert(0,'id',id)
        return datos
    else:
        return {}

In [25]:
def descargar_imagen(url:str, ruta_guardado:str):
    try:
        # Realizar la solicitud HTTP para obtener el contenido de la imagen
        respuesta = requests.get(url)
        
        # Extraer el nombre del archivo de la URL
        nombre_archivo = os.path.basename(url)

        # Construir la ruta completa para guardar la imagen
        ruta_completa = os.path.join(ruta_guardado, nombre_archivo)

        # Guardar el contenido de la imagen en un archivo local
        with open(ruta_completa, "wb") as archivo_local:
            archivo_local.write(respuesta.content)
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar la imagen: {e}")

In [26]:
def flujo_completo(id:int):
    try:
        crear_carpeta_en_datos(id)
        datos = obtener_datos_pokemon(id)
        url_imagen = datos['image'].values[0]
        descargar_imagen(url_imagen, f"datos/{id:03d}")
        datos.to_parquet(f"datos/{id:03d}/{id:03d}.parquet")
        return True
    except Exception as e:
        print(f"Error al obtener los datos del pokemon: {e}")
        return False





In [29]:
make_pool(flujo_completo, [(i,) for i in range(1, 252)], 100)

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [30]:
datos = pd.concat(make_pool(obtener_datos_pokemon, [(i,) for i in range(1, 252)], 100),ignore_index=True)

In [34]:
credentials = ServiceAccountCredentials.from_json_keyfile_name('creds/tsc20241-436644aacdef.json', 
['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])

client = gspread.authorize(credentials)
spreadsheet = client.open_by_key('1kNwp2JzT12s8-AmZx0FWaM5cMoAPILhrtgmzFKoGxjE')
worksheet = spreadsheet.get_worksheet(0)  
worksheet.update([datos.columns.tolist()] + datos.values.tolist())

/home/jose/entornos/tsc/lib/python3.10/site-packages/gspread/worksheet.py:1046: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


{'spreadsheetId': '1kNwp2JzT12s8-AmZx0FWaM5cMoAPILhrtgmzFKoGxjE',
 'updatedRange': "'Hoja 1'!A1:K252",
 'updatedRows': 252,
 'updatedColumns': 11,
 'updatedCells': 2772}

In [32]:
d = {'uno':1,'dos':2,'tres':3}

In [37]:
d.pop('dos')

2

In [38]:
d

{'uno': 1, 'tres': 3}